In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv('Data/AAPL_stock_data.csv', usecols=['open', 'close', 'high', 'low', 'volume'])
dataStock = dataset.copy()
dataStock = dataStock.dropna()

In [3]:
import sklearn
import sklearn.preprocessing as processing
dataStockNorm = dataStock.copy()
properties = ['open', 'close', 'high', 'low', 'volume']
scalerMinMax = processing.MinMaxScaler()
for i in properties:
    dataStockNorm[i] = scalerMinMax.fit_transform(dataStockNorm[i].values.reshape(-1, 1))

In [4]:
validPercentage = 0.1
testPercentage = 0.1
sequenceLength = 20
dataStockNorm_np = dataStockNorm.to_numpy()
dataSequence = []
for index in range(len(dataStockNorm_np) - sequenceLength):
    dataSequence.append(dataStockNorm_np[index : index+sequenceLength, :])
dataSequence = np.array(dataSequence)
validSize = int(np.round(validPercentage * dataSequence.shape[0]))
testSize = int(np.round(testPercentage * dataSequence.shape[0]))
trainSize = dataSequence.shape[0] - validSize - testSize
print(dataSequence.shape)
xTrain = dataSequence[:trainSize, :-1, :]
yTrain = dataSequence[:trainSize, -1, :]
xValid = dataSequence[trainSize : trainSize+validSize, :-1, :]
yValid = dataSequence[trainSize : trainSize+validSize, -1, :]
xTest = dataSequence[trainSize+validSize:, :-1, :]
yTest = dataSequence[trainSize+validSize:, -1, :]
print(xTrain.shape)
print(yTrain.shape)
print(yValid.shape)
print(xTest.shape)
xTrain = torch.tensor(xTrain, dtype=torch.float32)
yTrain = torch.tensor(yTrain, dtype=torch.float32)
xValid = torch.tensor(xValid, dtype=torch.float32)
yValid = torch.tensor(yValid, dtype=torch.float32)
xTest = torch.tensor(xTest, dtype=torch.float32)
yTest = torch.tensor(yTest, dtype=torch.float32)

(6498, 20, 5)
(5198, 19, 5)
(5198, 5)
(650, 5)
(650, 19, 5)


In [32]:
import NeuralNetworks.lstmStocks as lstmStocks
inputSize = 5
hiddenSize = 256
layers = 2
outputSize = 5
steps = sequenceLength - 1
criterion = nn.MSELoss(reduction='mean')
trainDataset = TensorDataset(xTrain, yTrain)
validDataset = TensorDataset(xValid, yValid)
trainLoader = DataLoader(trainDataset, batch_size=50, shuffle=True, num_workers=2)

model = lstmStocks.StockLSTMModel(inputSize, hiddenSize, layers, outputSize, steps)
optimizer = lstmStocks.StockLSTMOptimization(model, trainLoader, validDataset, criterion, 'cpu')
optimizer.optimizerAdam()

In [34]:
for i in range(100):
    train_loss = optimizer.trainEpoch()
    valid_loss = optimizer.validEpoch()
    print(f'Epoch [{i}], '
        f'Train Loss: {train_loss:.6f}, '
        f'Val Loss: {valid_loss:.6f}, ')

Epoch [0], Train Loss: 0.001311, Val Loss: 0.395574, 
Epoch [1], Train Loss: 0.001064, Val Loss: 0.045592, 
Epoch [2], Train Loss: 0.001011, Val Loss: 0.045558, 
Epoch [3], Train Loss: 0.000927, Val Loss: 0.096996, 
Epoch [4], Train Loss: 0.000865, Val Loss: 0.249692, 
Epoch [5], Train Loss: 0.000816, Val Loss: 0.029086, 
Epoch [6], Train Loss: 0.000777, Val Loss: 0.070546, 
Epoch [7], Train Loss: 0.000742, Val Loss: 0.071019, 
Epoch [8], Train Loss: 0.000737, Val Loss: 0.089521, 
Epoch [9], Train Loss: 0.000696, Val Loss: 0.109424, 
Epoch [10], Train Loss: 0.000679, Val Loss: 0.119438, 
Epoch [11], Train Loss: 0.000676, Val Loss: 0.048180, 
Epoch [12], Train Loss: 0.000664, Val Loss: 0.122540, 
Epoch [13], Train Loss: 0.000653, Val Loss: 0.034614, 


KeyboardInterrupt: 